# <center> Introduction to Spark In-memory Computing via Python PySpark </center>

In [1]:
import sys
import os
import pyspark

print(os.environ['SPARK_ROOT'])
print(os.environ['SPARK_CONFIG_FILE'])
print(os.environ['SPARK_ROOT'])
print(os.environ['SPARK_MASTER_HOST'])
print(os.environ['SPARK_MASTER_PORT'])
print(os.environ['SPARK_MASTER_WEBUI_PORT'])

/software/spackages/linux-rocky8-x86_64/gcc-9.5.0/spark-3.1.1-3takuhnpd3av65aoge5ark5gligt6usb
/home/amann3/ondemand/data/sys/dashboard/batch_connect/sys/ood_jupyter_spark/output/04a1b626-170a-4ead-8f11-0b0661301574/spark-defaults.conf
/software/spackages/linux-rocky8-x86_64/gcc-9.5.0/spark-3.1.1-3takuhnpd3av65aoge5ark5gligt6usb
node0655.palmetto.clemson.edu
6304
3727


In [2]:
!cp -R /zfs/citi/movielens /scratch1/$USER/

In [9]:
!ls -lh /scratch1/$USER/movielens

total 320M
-rw-r--r-- 1 amann3 cuuser 318M May 31 11:18 genome-scores.csv
-rw-r--r-- 1 amann3 cuuser  18K May 31 11:18 genome-tags.csv
-rw-r--r-- 1 amann3 cuuser 840K May 31 11:18 links.csv
-rw-r--r-- 1 amann3 cuuser 2.0M May 31 11:18 movies.csv
-rw-r--r-- 1 amann3 cuuser 633M May 31 11:18 ratings.csv
-rw-r--r-- 1 amann3 cuuser 9.3K May 31 11:18 README.txt
-rw-r--r-- 1 amann3 cuuser  23M May 31 11:18 tags.csv


In [4]:
!cat /scratch1/$USER/movielens/README.txt

Summary

This dataset (ml-latest) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 24404096 ratings and 668953 tag applications across 40110 movies. These data were created by 259137 users between January 09, 1995 and October 17, 2016. This dataset was generated on October 18, 2016.

Users were selected at random for inclusion. All selected users had rated at least 1 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files `genome-scores.csv`, `genome-tags.csv`, `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This is a *development* dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available *benchmark* datasets if that is your intent.

This and other GroupLens da

In [5]:
!cat /scratch1/$USER/movielens/links.csv \
    2>/dev/null | head -n 5

movieId,imdbId,tmdbId
1,0114709,862
2,0113497,8844
3,0113228,15602
4,0114885,31357


In [6]:
!cat /scratch1/$USER/movielens/movies.csv \
    2>/dev/null | head -n 5

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance


In [7]:
!cat /scratch1/$USER/movielens/ratings.csv \
    2>/dev/null | head -n 5

userId,movieId,rating,timestamp
1,122,2.0,945544824
1,172,1.0,945544871
1,1221,5.0,945544788
1,1441,4.0,945544871


In [8]:
!cat /scratch1/$USER/movielens/tags.csv \
    2>/dev/null | head -n 5

userId,movieId,tag,timestamp
28,63062,angelina jolie,1263047558
40,4973,Poetic,1436439070
40,117533,privacy,1436439140
57,356,life positive,1291771526


In [10]:
import os
uid = os.getenv('USER')
rating_file = "/scratch1/" + uid + "/movielens/ratings.csv"
ratings = sc.textFile(rating_file)

In [11]:
ratings.cache()

/scratch1/amann3/movielens/ratings.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [12]:
%%time
ratings.count()

CPU times: user 18.2 ms, sys: 20.6 ms, total: 38.8 ms
Wall time: 30.7 s


24404097

In [13]:
%%time
ratings.count()

CPU times: user 14.2 ms, sys: 7.26 ms, total: 21.5 ms
Wall time: 5.07 s


24404097

In [14]:
%%time
ratings.count()

CPU times: user 10.4 ms, sys: 10.2 ms, total: 20.6 ms
Wall time: 5.23 s


24404097

### Find movies which have the highest average ratings over the years and identify the corresponding genre.

- Find the average ratings of all movies over the years
- Identify the corresponding genres for each movie

In [15]:
ratings.take(5)

['userId,movieId,rating,timestamp',
 '1,122,2.0,945544824',
 '1,172,1.0,945544871',
 '1,1221,5.0,945544788',
 '1,1441,4.0,945544871']

In [16]:
ratingHeader = ratings.first() #extract header
print(ratingHeader)

userId,movieId,rating,timestamp


In [17]:
ratingsOnly = ratings.filter(lambda x: x != ratingHeader)

In [18]:
ratingsOnly.take(5)

['1,122,2.0,945544824',
 '1,172,1.0,945544871',
 '1,1221,5.0,945544788',
 '1,1441,4.0,945544871',
 '1,1609,3.0,945544824']

In [19]:
movieRatings = ratingsOnly.map(lambda line: (line.split(",")[1], float(line.split(",")[2])))

In [20]:
movieRatings.take(5)

[('122', 2.0), ('172', 1.0), ('1221', 5.0), ('1441', 4.0), ('1609', 3.0)]

**Possible approaches in aggregating data:** 
- groupByKey and mapValues
- reduceByKey and countByKey

**groupByKey and mapValues**

In [21]:
groupByKeyRatings = movieRatings.groupByKey()
groupByKeyRatings.take(5)

[('356', <pyspark.resultiterable.ResultIterable at 0x146fb276e2e0>),
 ('464', <pyspark.resultiterable.ResultIterable at 0x146fb276e460>),
 ('520', <pyspark.resultiterable.ResultIterable at 0x146fb276ee50>),
 ('1013', <pyspark.resultiterable.ResultIterable at 0x146fb276e4f0>),
 ('1129', <pyspark.resultiterable.ResultIterable at 0x146fb276ed30>)]

In [22]:
mapValuesToListRatings = groupByKeyRatings.mapValues(list)
mapValuesToListRatings.take(5)

[('356',
  [3.0,
   4.5,
   5.0,
   4.0,
   5.0,
   1.0,
   5.0,
   5.0,
   5.0,
   2.0,
   3.0,
   3.0,
   3.0,
   5.0,
   5.0,
   2.0,
   3.5,
   4.0,
   5.0,
   4.5,
   5.0,
   4.0,
   5.0,
   4.0,
   5.0,
   4.5,
   3.0,
   4.0,
   4.0,
   4.0,
   4.5,
   5.0,
   2.5,
   5.0,
   3.0,
   4.0,
   5.0,
   2.0,
   4.0,
   4.0,
   4.5,
   3.0,
   5.0,
   4.0,
   5.0,
   5.0,
   5.0,
   4.0,
   1.0,
   4.0,
   5.0,
   3.0,
   4.0,
   5.0,
   2.5,
   5.0,
   3.0,
   5.0,
   3.5,
   5.0,
   5.0,
   4.0,
   4.0,
   3.0,
   4.0,
   4.0,
   2.0,
   4.0,
   4.5,
   3.0,
   5.0,
   3.0,
   3.0,
   4.5,
   3.0,
   4.0,
   3.0,
   5.0,
   1.0,
   5.0,
   4.0,
   5.0,
   4.5,
   5.0,
   5.0,
   5.0,
   3.0,
   4.0,
   5.0,
   4.5,
   2.0,
   5.0,
   3.0,
   4.5,
   5.0,
   2.0,
   4.0,
   5.0,
   5.0,
   2.0,
   5.0,
   3.0,
   4.0,
   4.5,
   3.0,
   5.0,
   4.5,
   5.0,
   3.5,
   4.0,
   5.0,
   2.0,
   5.0,
   2.0,
   3.0,
   3.5,
   4.0,
   5.0,
   5.0,
   4.0,
   4.5,
   5.0,
   3.0,
   5.0,

In [23]:
avgRatings01 = mapValuesToListRatings.mapValues(lambda V: sum(V) / float(len(V)))

avgRatings01.take(5)

[('356', 4.047108935806716),
 ('464', 2.922174840085288),
 ('520', 3.057137663596704),
 ('1013', 3.407130367204197),
 ('1129', 3.367892644135189)]

Is this correct?

In [24]:
test = [2.0, 4.0, 3.0]
sum(test) / len(test)

3.0

**reduceByKey and countByKey**

In [25]:
countsByKey = movieRatings.countByKey()

countsByKey

defaultdict(int,
            {'122': 4060,
             '172': 14078,
             '1221': 34508,
             '1441': 6723,
             '1609': 780,
             '1961': 29790,
             '1972': 1534,
             '441': 9386,
             '494': 14299,
             '1193': 37995,
             '1597': 12262,
             '1608': 16097,
             '1641': 16953,
             '2628': 34632,
             '3454': 420,
             '3519': 1087,
             '4963': 28770,
             '1': 63469,
             '5': 15023,
             '11': 19281,
             '21': 25337,
             '32': 51931,
             '34': 34569,
             '61': 2441,
             '79': 6288,
             '107': 7982,
             '140': 5623,
             '150': 56077,
             '158': 15206,
             '161': 25761,
             '165': 39597,
             '185': 26301,
             '225': 13294,
             '236': 13426,
             '237': 6712,
             '252': 10830,
             '256': 11

In [26]:
def sumValues(x,y):
    return (x + y)

sumRatings = movieRatings.reduceByKey(sumValues)

sumRatings.take(5)

[('356', 350597.0),
 ('464', 9593.5),
 ('520', 50455.0),
 ('1013', 17536.5),
 ('1129', 33881.0)]

In [27]:
import operator

sumRatings = movieRatings.reduceByKey(operator.add)
sumRatings.take(5)

[('356', 350597.0),
 ('553', 59427.0),
 ('65261', 9253.0),
 ('3913', 413.0),
 ('2959', 230004.5)]

In [28]:
avgRatings02 = sumRatings.map(lambda x: (x[0], x[1] / countsByKey.get(x[0])))

avgRatings02.take(5)

[('356', 4.047108935806716),
 ('464', 2.922174840085288),
 ('520', 3.057137663596704),
 ('1013', 3.407130367204197),
 ('1129', 3.367892644135189)]

How do we augment movie ratings data with title information?

In [29]:
import os
uid = os.getenv('USER')
movie_file = "/scratch1/" + uid + "/movielens/movies.csv"
movies = sc.textFile(movie_file)

In [30]:
movieHeader = movies.first() #extract header
print(movieHeader)

movieId,title,genres


In [31]:
movies = movies.filter(lambda x: x != movieHeader)

movies.take(50)

['1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 '2,Jumanji (1995),Adventure|Children|Fantasy',
 '3,Grumpier Old Men (1995),Comedy|Romance',
 '4,Waiting to Exhale (1995),Comedy|Drama|Romance',
 '5,Father of the Bride Part II (1995),Comedy',
 '6,Heat (1995),Action|Crime|Thriller',
 '7,Sabrina (1995),Comedy|Romance',
 '8,Tom and Huck (1995),Adventure|Children',
 '9,Sudden Death (1995),Action',
 '10,GoldenEye (1995),Action|Adventure|Thriller',
 '11,"American President, The (1995)",Comedy|Drama|Romance',
 '12,Dracula: Dead and Loving It (1995),Comedy|Horror',
 '13,Balto (1995),Adventure|Animation|Children',
 '14,Nixon (1995),Drama',
 '15,Cutthroat Island (1995),Action|Adventure|Romance',
 '16,Casino (1995),Crime|Drama',
 '17,Sense and Sensibility (1995),Drama|Romance',
 '18,Four Rooms (1995),Comedy',
 '19,Ace Ventura: When Nature Calls (1995),Comedy',
 '20,Money Train (1995),Action|Comedy|Crime|Drama|Thriller',
 '21,Get Shorty (1995),Comedy|Crime|Thriller',
 '22,Copycat 

In [32]:
s = '40,"Cry, the Beloved Country (1995)",Drama'
s.rsplit(",",1)[0]

'40,"Cry, the Beloved Country (1995)"'

In [33]:
s.rsplit(",",1)[0].split(",",1)[1]

'"Cry, the Beloved Country (1995)"'

**NOTE:** This is not a good way to handle CSV parsing since some strings may contain commas. Instead Consider using a CSV library supported by Spark. We will show this in the next notebook in the series. The following will work for now but is difficult to understand.

In [34]:
movieInfo = movies.map(lambda line: (line.split(",")[0], ((line.rsplit(",",1)[0]).split(",",1)[1], line.rsplit(",",1)[1])))

movieInfo.take(50)

[('1', ('Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy')),
 ('2', ('Jumanji (1995)', 'Adventure|Children|Fantasy')),
 ('3', ('Grumpier Old Men (1995)', 'Comedy|Romance')),
 ('4', ('Waiting to Exhale (1995)', 'Comedy|Drama|Romance')),
 ('5', ('Father of the Bride Part II (1995)', 'Comedy')),
 ('6', ('Heat (1995)', 'Action|Crime|Thriller')),
 ('7', ('Sabrina (1995)', 'Comedy|Romance')),
 ('8', ('Tom and Huck (1995)', 'Adventure|Children')),
 ('9', ('Sudden Death (1995)', 'Action')),
 ('10', ('GoldenEye (1995)', 'Action|Adventure|Thriller')),
 ('11', ('"American President, The (1995)"', 'Comedy|Drama|Romance')),
 ('12', ('Dracula: Dead and Loving It (1995)', 'Comedy|Horror')),
 ('13', ('Balto (1995)', 'Adventure|Animation|Children')),
 ('14', ('Nixon (1995)', 'Drama')),
 ('15', ('Cutthroat Island (1995)', 'Action|Adventure|Romance')),
 ('16', ('Casino (1995)', 'Crime|Drama')),
 ('17', ('Sense and Sensibility (1995)', 'Drama|Romance')),
 ('18', ('Four Rooms (1995)', 'Come

In [35]:
augmentedRatings = avgRatings01.join(movieInfo)

augmentedRatings.take(5)

[('2959',
  (4.229890025011034, ('Fight Club (1999)', 'Action|Crime|Drama|Thriller'))),
 ('1198',
  (4.15006599835896,
   ('Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
    'Action|Adventure'))),
 ('69524',
  (3.9783174327840416,
   ('Raiders of the Lost Ark: The Adaptation (1989)',
    'Action|Adventure|Thriller'))),
 ('633', (3.3864942528735633, ('Denise Calls Up (1995)', 'Comedy'))),
 ('752',
  (2.909090909090909,
   ('Vermont Is For Lovers (1992)', 'Comedy|Documentary|Romance')))]

In [36]:
x = ('2959',
  (4.229890025011034, ('Fight Club (1999)', 'Action|Crime|Drama|Thriller')))

In [37]:
x[0]

'2959'

In [38]:
x[1]

(4.229890025011034, ('Fight Club (1999)', 'Action|Crime|Drama|Thriller'))

In [39]:
x[1][0]

4.229890025011034

*Movie with highest average rating:*

In [40]:
augmentedRatings.takeOrdered(10, key = lambda x : -x[1][0])

[('159423',
  (5.0,
   ('Jonas Brothers: The Concert Experience (2009)', '(no genres listed)'))),
 ('112674', (5.0, ('Bulletproof Salesman (2008)', 'Documentary'))),
 ('164869', (5.0, ('A Cinderella Story: If the Shoe Fits (2016)', 'Comedy'))),
 ('160325', (5.0, ('I Love Hong Kong (2011)', 'Comedy'))),
 ('114353', (5.0, ('Heavyweights (Schwere Jungs) (2006)', 'Comedy'))),
 ('123727', (5.0, ('Immigration Tango (2011)', 'Comedy|Romance'))),
 ('93967',
  (5.0,
   ('"Keeping the Promise (Sign of the Beaver, The) (1997)"',
    'Children|Drama'))),
 ('133575', (5.0, ('Do Detectives Think? (1927)', 'Comedy'))),
 ('145939', (5.0, ('Sandesham (1991)', 'Children|Comedy'))),
 ('146946', (5.0, ('The Hardy Bucks Movie (2013)', 'Comedy')))]

*Movie with lowest average rating:*

In [41]:
augmentedRatings.takeOrdered(10, key = lambda x : x[1][0])

[('154582', (0.5, ('The Dark Diamond (2004)', '(no genres listed)'))),
 ('145285', (0.5, ('Octopus (2000)', 'Action|Horror|Thriller'))),
 ('117172', (0.5, ('BFFs (2014)', 'Comedy'))),
 ('133541', (0.5, ('Two Hundred Thousand Dirty (2014)', 'Comedy'))),
 ('138008', (0.5, ('New Year (2011)', '(no genres listed)'))),
 ('143691', (0.5, ('Criminally Insane 2 (1987)', 'Horror'))),
 ('80154', (0.5, ('Urban Menace (1999)', 'Action|Horror'))),
 ('144966', (0.5, ('Alive (2006)', 'Drama|Fantasy|Mystery'))),
 ('131152', (0.5, ('The Fat Spy (1966)', 'Comedy'))),
 ('73196',
  (0.5, ('My Love Has Been Burning (Waga koi wa moenu) (1949)', 'Drama')))]

### Challenge:

Make appropriate changes so that only movies with average ratings higher than 3.75 and number of ratings totalling at least 1000 are collected.

In [42]:
highRatings = sumRatings.filter(lambda x: countsByKey.get(x[0]) >= 1000)
print(highRatings.take(10))

avgHighRatings = highRatings.map(lambda x: (x[0], x[1] / countsByKey.get(x[0])))
avgHighRatings = avgHighRatings.filter(lambda x: x[1] > 3.75)
augmentedHighRatings = avgHighRatings.join(movieInfo)

augmentedHighRatings.takeOrdered(10, key = lambda x : -x[1][0])

[('356', 350597.0), ('553', 59427.0), ('65261', 9253.0), ('2959', 230004.5), ('5952', 192739.0), ('10', 109817.0), ('50', 242783.0), ('520', 50455.0), ('1196', 239052.0), ('1198', 232661.0)]


[('318',
  (4.43308862707951, ('"Shawshank Redemption, The (1994)"', 'Crime|Drama'))),
 ('858', (4.343623358918333, ('"Godfather, The (1972)"', 'Crime|Drama'))),
 ('50',
  (4.308635621494996,
   ('"Usual Suspects, The (1995)"', 'Crime|Mystery|Thriller'))),
 ('527', (4.2759629983252205, ("Schindler's List (1993)", 'Drama|War'))),
 ('1221',
  (4.263098411962443, ('"Godfather: Part II, The (1974)"', 'Crime|Drama'))),
 ('2019',
  (4.256196804539346,
   ('Seven Samurai (Shichinin no samurai) (1954)', 'Action|Adventure|Drama'))),
 ('94466', (4.2426322163907955, ('Black Mirror (2011)', 'Drama|Sci-Fi'))),
 ('904', (4.238590226483393, ('Rear Window (1954)', 'Mystery|Thriller'))),
 ('1193',
  (4.232701671272536, ("One Flew Over the Cuckoo's Nest (1975)", 'Drama'))),
 ('2959',
  (4.229890025011034, ('Fight Club (1999)', 'Action|Crime|Drama|Thriller')))]

### Find genres which have the highest average ratings over the years

- Identify the genres associated with a movie and its rating
- Each movie can have multiple genres. How to flip the Key/Value pair?

In [ ]:
movieRatings.take(5)

In [ ]:
movieInfo.take(5)

In [ ]:
augmentedInfo = movieRatings.join(movieInfo)

In [ ]:
augmentedInfo.take(5)

In [ ]:
def extractGenreRating (t):
    final_tuples = []
    genreList = t[1][1][1].split("|")
    for genre in genreList:
        final_tuples.append((genre,t[1][0]))
    return final_tuples

print(extractGenreRating((u'1', (3.0, (u'Toy Story (1995)', u'Adventure|Animation|Children|Comedy|Fantasy')))))

In [ ]:
def countGenre (t):
    genreList = t[1][1][1].split("|")
    return (t[0],(len(genreList), t[1][0]))

print(countGenre(('112674', (5.0, ('Bulletproof Salesman (2008)', 'Documentary')))))

In [ ]:
genreCount = augmentedRatings.map(countGenre)
genreCount.take(10)

In [ ]:
genreStudy = avgRatings01.join(movieInfo)

In [ ]:
genreRatings = augmentedInfo.flatMap(extractGenreRating)

In [ ]:
genreRatings.take(20)

In [ ]:
countsByKey = genreRatings.countByKey()

countsByKey

In [ ]:
genreRatings.take(5)

### Challenge:

Calculate the average rating for each genre